In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras import models
import keras
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing import image

In [2]:
# Netzwerkarchitektur
model = Sequential()

    # Detektion
model.add(Conv2D(32, kernel_size=(5,5), activation= 'relu', input_shape= mnist_inputshape))

    # Conv_Block 2
model.add(Conv2D(64, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

    # Identifikation
model.add(Flatten())
model.add(Dense(128, activation='relu', name='features'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

NameError: name 'Sequential' is not defined

In [ ]:
# Festlegung der Verlustfunktion und Optimierung
model.compile(loss='categorial_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
# Trainig des Models
LOGDIR = "D:/logs"
tensorboard = TensorBoard(log_dir=LOGDIR, histogram_freq=0,
                             write_graph=True,
                             write_images=True)

# Hyperparameter
batch_size = 128
num_classes = 10
epochs = 12

history = model.fit(train_images, train_labels,
                    batch_size=batch_size,
                    callbacks=[tensorboard],
                    epochs=epochs,
                    verbose=1,
                    validation_data=(test_images, test_labels))

In [ ]:
# Evaluation
score = model.evaluate(test_images, test_labels)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

In [ ]:
# Speichern
model.save('Model.h5')

In [ ]:
' Laden des zu klassifizierenden Bildes'
path = ''
file = 'vogel.jpg'
img_file = os.path.join(path,file)

# Transformation in Bildgröße
img = image.load_img(img_file, target_size =(229,229))

x = image.img_to_array(img)
print(x.shape)
print(x)

In [ ]:
# zusätzliche Dimension für die Batchverarbeitung der Bilder
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print(x.shape)
print(x)

In [ ]:
# Klassifikation des Bildes
prediction = model.predict(x)

decoded = decode_predictions(prediction, top=3)[0]
print(decoded)

In [ ]:
# klassifiziertes Bild mit Klassenlegende
plt.figure(figsize=(8,8))

pos_x, pos_y = 1, 10
for id, lab, prob in decoded:
    plt.text(pos_x, pos_y, 'Klasse: {} - {:.2f}%'.format(lab, prob*100),
             fontsize=12, color='black',
             bbox=dict(boxstyle='round', pad=0.1, fc='white'))
    pos_y += 10